%load_ext autoreload
%autoreload 2

In [1]:
import requests
import aiohttp
import random
import json
import uuid
import asyncio
import os

import numpy as np
from common.models.trial import Solution, Advisor, WrittenStrategy, Trial
from utils.process import process_solution

from dotenv import load_dotenv

In [2]:

# create new experiment
max_participants = 50  # Number of parallel participants in simulation
experiment_type = 'sim_3states_no_social_v3'
local = True
social_learning_strategy = "random"

load_dotenv()

if local:
    BACKEND_URL = 'http://localhost:5050'
    BACKEND_USER = 'admin'
    BACKEND_PASSWORD = 'admin'
else:
    BACKEND_USER = os.environ['BACKEND_USER']
    BACKEND_PASSWORD = os.environ['BACKEND_PASSWORD']
    BACKEND_URL = os.environ['BACKEND_URL']


In [3]:
url = f"{BACKEND_URL}/admin/config"
auth = (BACKEND_USER, BACKEND_PASSWORD)


payload = json.dumps({
  "active": True,
  "created_at": "2023-10-17T09:57:36.204000",
  "redirect_url": "https://app.prolific.co/submissions/complete",
  "experiment_type": experiment_type,
  "rewrite_previous_data": True,
  "seed": 1,
  "n_generations": 5,
  "n_ai_players": 3,
  "networks_path": "data/24_02_04",
  "n_sessions_per_generation": 16,
  "n_advise_per_session": 5,
  "n_session_tree_replications": 4,
  "conditions": [
    "w_ai",
    "wo_ai"
  ],
  "n_social_learning_blocks": 1,
  "n_social_learning_networks_per_block": 4,
  "n_practice_trials": 2,
  "n_demonstration_trials": 4,
  "simulate_humans": False,
  "social_learning_trials": [
    "observation",
    "repeat",
    "try_yourself"
  ],
  "main_only": True,
  "session_timeout": 10,
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Basic YWRtaW46YWRtaW4='
}

response = requests.request("POST", url, headers=headers, data=payload, auth=auth)

print(response.text)

{"id":"65faaf17977538d15bd3a722","active":true,"created_at":"2024-03-20T09:40:39.902000","redirect_url":"https://app.prolific.co/submissions/complete","experiment_type":"sim_3states_no_social_v3","rewrite_previous_data":true,"networks_path":"data/24_02_04","seed":1,"n_generations":5,"n_ai_players":3,"n_sessions_per_generation":16,"n_advise_per_session":5,"n_session_tree_replications":4,"conditions":["w_ai","wo_ai"],"n_social_learning_blocks":1,"n_social_learning_networks_per_block":4,"n_practice_trials":2,"n_demonstration_trials":4,"simulate_humans":false,"social_learning_trials":["observation","repeat","try_yourself"],"main_only":true,"session_timeout":10.0,"shared_first_generation":false}


In [4]:
import aiohttp
import time

session = aiohttp.ClientSession()

async def get_trial(prolific_id, experiment_type):
    url = f"{BACKEND_URL}/session/{experiment_type}/{prolific_id}"
    headers = {'Content-Type': 'application/json'}

    start_time = time.time()  # Start the timer

    async with session.get(url, headers=headers) as response:
        response_time = time.time() - start_time  # Calculate the response time

        if response.status == 200:
            try:
                trial = Trial(**await response.json())
                return trial, response_time
            except:
                return None, response_time
        else:
            return None, response_time


async def post_trial(prolific_id, trial_id, body):
    url = f"{BACKEND_URL}/session/{prolific_id}/{trial_id}"
    headers = {'Content-Type': 'application/json'}

    start_time = time.time()  # Start the timer

    if body is not None:
        async with session.post(url, headers=headers, data=body) as response:
            response_time = time.time() - start_time  # Calculate the response time
            return response.status == 200, response_time
    else:
        async with session.post(url, headers=headers) as response:
            response_time = time.time() - start_time  # Calculate the response time
            return response.status == 200, response_time


In [5]:
from pathlib import Path
from common.utils.utils import estimate_solution_score, estimate_average_player_score
from common.models.network import Network


networks_path = Path("../data/24_02_04")
network_data = json.load(open(networks_path / "networks.json"))
solutions_myopic = json.load(open(networks_path / "solution__myopic.json"))
solutions_m1 = json.load(open(networks_path / "machine_solutions" / "0.json"))
solutions_random = json.load(open(networks_path / "solution__random.json"))
networks_by_id = {n["network_id"]: n for n in network_data}
solutions_myopic_by_id = {s["network_id"]: s for s in solutions_myopic}
solutions_m1_by_id = {s["network_id"]: s for s in solutions_m1}
solutions_random_by_id = {s["network_id"]: s for s in solutions_random}


def _get_solution(network_id, solution_type):
    network = networks_by_id[network_id]
    # get the solution for the network
    if solution_type == "myopic":
        solution = solutions_myopic_by_id[network_id]
    elif solution_type == "machine":
        solution = solutions_m1_by_id[network_id]
    elif solution_type == "random":
        solution = solutions_random_by_id[network_id]
    else:
        raise ValueError("Invalid solution type")

    solution['moves'][0] = network['starting_node']
    score = estimate_solution_score(Network(**network), solution['moves'], 10)
    assert score != 100_000, f"Invalid solution score: {score} for solution: {solution_type} and network: {network_id}"

    return Solution(**solution)


def get_solution(network_id, state):
    assert np.absolute(state.sum() - 1) < 0.0001, f"Invalid state: {state}"
    assert np.all(state >= 0), f"Invalid state: {state}"
    s_type_idx = np.random.choice(list(range(len(state))), p=state)
    s_type = ['random', "myopic", "machine"][s_type_idx]
    return _get_solution(network_id, s_type)


def get_solution_evaluation(solution: Solution, network_id):
    # get rewards
    evaluation = process_solution(networks_by_id[network_id], solution.dict())
    return evaluation

In [6]:
p_mypopic_init = 0.3
max_p = 1.
drop_rate = 0.002
incomplete_rate = 0.002
average_time_per_trial = 0

individual_learning_factors = (
    (0.001, [0,0,1]),
    (0.2, [0,1,0]),
    (0.799, [0,0,0])
)

social_learning_factors = {
    'optimal': (
        (0.11, [0,0,1]),
        (0.89, [0,0,0])
    ),
    'myopic': (
        (0.11, [0,1,0]),
        (0.89, [0,0,0])
    )
}



def scale_state(state, change):
    state = state + change
    state[2] = np.minimum(state[2], max_p)
    state[1] = np.minimum(state[1], np.minimum(1 - state[2], max_p))
    state[0] = 1 - state[2] - state[1]
    assert np.absolute(state.sum() - 1) < 0.0001, f"Invalid state: {state}"
    assert np.all(state >= 0), f"Invalid state: {state}"
    return state


def sample_change(options):
    p = np.array([o[0] for o in options])
    assert np.all(p >= 0), f"Invalid probabilities: {p}"
    assert np.absolute(p.sum() - 1) < 0.0001, f"Invalid probabilities: {p}"
    values = np.array([o[1] for o in options])
    change_idx = np.random.choice(list(range(len(p))), p=p)
    change = values[change_idx]
    return change


def individual_learning(state):
    change = sample_change(individual_learning_factors)
    state = scale_state(state, change)
    return state


def social_learning(state, strategy):
    change = sample_change(social_learning_factors[strategy])
    state = scale_state(state, change)
    return state

def init_state():
    assert p_mypopic_init <= 0.5, f"Invalid p_mypopic_init: {p_mypopic_init}"
    p_mypopic = random.random() * p_mypopic_init * 2
    state = np.array([1 - p_mypopic, p_mypopic, 0]) # [random, myopic, machine]
    return state

def handle_instruction_trial(trial, state):
    body = None
    return body, state

def handle_individual_trial(trial, state):
    solution = get_solution(trial.network.network_id, state)
    state = individual_learning(state)
    return solution.json(), state

def handle_written_strategy_trial(trial, state):
    strategy = WrittenStrategy(
        strategy=''
    )
    body = strategy.json()
    return body, state

def handle_demonstration_trial(trial, state):
    solution = get_solution(trial.network.network_id, state)
    if random.random() < incomplete_rate:
        solution.moves = solution.moves[:-random.randint(1, 3)]
    return solution.json(), state

def handle_debriefing_trial(trial, state):
    body = None
    return body, state

def handle_social_learning_selection_trial(trial, state):
    advisor_selection = trial.advisor_selection
    advisor = advisor_selection.advisor_ids
    scores = advisor_selection.scores

    if social_learning_strategy == "best":
        # select the advisor with the highest score
        max_score_idx = np.argmax(scores)
        advisor = advisor[max_score_idx]
    elif social_learning_strategy == "random":
        # select a random advisor
        advisor = random.choice(advisor)
    else:
        raise ValueError(f"Invalid social learning strategy: {social_learning_strategy}")

    selection = Advisor(
        advisor_id=advisor
    )
    body = selection.json()
    return body, state

def handle_observation_trial(trial, state):
    solution = trial.advisor.solution
    network_id = trial.network.network_id
    evaluation = get_solution_evaluation(solution, network_id)
    body = None
    return body, state

def handle_repeat_trial(trial, state):
    solution = trial.advisor.solution

    solution = trial.advisor.solution
    network_id = trial.network.network_id
    evaluation = get_solution_evaluation(solution, network_id)
    if evaluation['optimal'] == 10:
        state = social_learning(state, 'optimal')
    elif evaluation['myopic'] > 0:
        state = social_learning(state, 'myopic')

    body = solution.json()
    return body, state

def handle_try_yourself_trial(trial, state):
    solution = get_solution(trial.network.network_id, state)
    state = individual_learning(state)
    return solution.json(), state


def handle_trial(trial, state):
    if trial.trial_type == "instruction":
        return handle_instruction_trial(trial, state)
    elif trial.trial_type == "individual":
        return handle_individual_trial(trial, state)
    elif trial.trial_type == "written_strategy":
        return handle_written_strategy_trial(trial, state)
    elif trial.trial_type == "demonstration":
        return handle_demonstration_trial(trial, state)
    elif trial.trial_type == "debriefing":
        return handle_debriefing_trial(trial, state)
    elif trial.trial_type == "social_learning_selection":
        return handle_social_learning_selection_trial(trial, state)
    elif trial.trial_type == "observation":
        return handle_observation_trial(trial, state)
    elif trial.trial_type == "repeat":
        return handle_repeat_trial(trial, state)
    elif trial.trial_type == "try_yourself":
        return handle_try_yourself_trial(trial, state)
    else:
        raise ValueError(f"{trial.trial_type} is an invalid trial type")

In [7]:

async def run_participant():
    max_skip = 10
    trials = []
    prolific_id = "sim_" + uuid.uuid4().hex[:8]
    state = init_state()
    current_trial_id = None
    while True:
        trial, get_response_time = await get_trial(prolific_id, experiment_type)
        if trial is None:
            await asyncio.sleep(5)
            max_skip -= 1
            if max_skip < 0:
                break
            continue
        if trial.id == current_trial_id:
            if trial.trial_type == "debriefing":
                break
            else:
                raise ValueError(f"Trial {trial.id} of type {trial.trial_type} of prolific id {prolific_id} is a duplicate")

        if random.random() < drop_rate / 2:
            trials = [{**t, 'dropped': True} for t in trials]
            break

        current_trial_id = trial.id
        body, state = handle_trial(trial, state)

        sleep_time = average_time_per_trial * (1 + random.random() - 0.5)
        sleep_time = np.maximum(sleep_time, 0)
        await asyncio.sleep(sleep_time)

        _, post_response_time = await post_trial(prolific_id, trial.id, body)
        trial_clean = json.loads(trial.json())
        # session_clean = json.loads(trail.session.json())
        trials.append({'trial': trial_clean, 'prolific_id': prolific_id, 'response_time': get_response_time, 'post_response_time': post_response_time})

        if random.random() < drop_rate / 2:
            trials = [{**t, 'dropped': True} for t in trials]
            break

    return trials

In [8]:
def save_trials(trials, experiment_type, max_participants):
    trials = [{**t, 'experiment_type': experiment_type, 'max_participants': max_participants} for t in trials]

    # Save trials as json
    trials_file = os.path.join('data', 'simulations', f'{experiment_type}.json')
    with open(trials_file, 'w') as f:
        json.dump(trials, f, indent=4)


In [9]:

def check_uncompleted(experiment_type):
    completed = False
    url = f'{BACKEND_URL}/results'
    headers = {'Accept': 'application/json'}
    auth = (BACKEND_USER, BACKEND_PASSWORD)
    sessions = requests.get(f'{url}/sessions?experiment_type={experiment_type}&completed={completed}', headers=headers, auth=auth)
    print(sessions, flush=True)
    sessions_json = sessions.json()

    n_uncompleted_sessions = len([s for s in sessions_json if (not s['finished'] and not s['expired'])])
    return n_uncompleted_sessions


async def run_with_limit(semaphore, trials):
    async with semaphore:
        new_trials = await run_participant()
        if new_trials:
            trials.extend(new_trials)


async def main(max_concurrent_tasks):
    trials = []
    semaphore = asyncio.Semaphore(max_concurrent_tasks)
    tasks = []

    n_loop = 0
    while True:
        n_sessions = check_uncompleted(experiment_type) - len(tasks)
        if n_sessions == 0:
            break

        n_completed = 0
        n_started = 0

        # Assuming you have a way to determine if more participants should be run
        while n_completed < n_sessions:
            while len(tasks) < max_concurrent_tasks and n_started < n_sessions:
                print(f"Loop {n_loop}, Started: {n_started}, Finished: {n_completed}, Remaining: {n_sessions - n_completed}, Running: {len(tasks)}, Total: {n_sessions}")
                task = asyncio.create_task(run_with_limit(semaphore, trials))
                n_started += 1
                tasks.append(task)

            # Wait for one of the tasks to complete
            done, _ = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)

            n_completed += len(done)

            # Remove the completed tasks
            tasks = [t for t in tasks if not t.done()]

            if len(trials) % 50 == 0:
                save_trials(trials, experiment_type, max_participants)
        n_loop += 1
    # Wait for the remaining tasks to complete
    if tasks:
        await asyncio.wait(tasks)

    return trials

# Run the main coroutine with the desired maximum number of concurrent tasks

trials = await main(max_participants)

save_trials(trials, experiment_type, max_participants)


<Response [200]>
Loop 0, Started: 0, Finished: 0, Remaining: 308, Running: 0, Total: 308
Loop 0, Started: 1, Finished: 0, Remaining: 308, Running: 1, Total: 308
Loop 0, Started: 2, Finished: 0, Remaining: 308, Running: 2, Total: 308
Loop 0, Started: 3, Finished: 0, Remaining: 308, Running: 3, Total: 308
Loop 0, Started: 4, Finished: 0, Remaining: 308, Running: 4, Total: 308
Loop 0, Started: 5, Finished: 0, Remaining: 308, Running: 5, Total: 308
Loop 0, Started: 6, Finished: 0, Remaining: 308, Running: 6, Total: 308
Loop 0, Started: 7, Finished: 0, Remaining: 308, Running: 7, Total: 308
Loop 0, Started: 8, Finished: 0, Remaining: 308, Running: 8, Total: 308
Loop 0, Started: 9, Finished: 0, Remaining: 308, Running: 9, Total: 308
Loop 0, Started: 10, Finished: 0, Remaining: 308, Running: 10, Total: 308
Loop 0, Started: 11, Finished: 0, Remaining: 308, Running: 11, Total: 308
Loop 0, Started: 12, Finished: 0, Remaining: 308, Running: 12, Total: 308
Loop 0, Started: 13, Finished: 0, Remaini

Task exception was never retrieved
future: <Task finished name='Task-11043' coro=<run_with_limit() done, defined at /tmp/ipykernel_980350/2588555149.py:14> exception=ServerDisconnectedError('Server disconnected')>
Traceback (most recent call last):
  File "/tmp/ipykernel_980350/2588555149.py", line 16, in run_with_limit
    new_trials = await run_participant()
  File "/tmp/ipykernel_980350/4172783153.py", line 8, in run_participant
    trial, get_response_time = await get_trial(prolific_id, experiment_type)
  File "/tmp/ipykernel_980350/1537036795.py", line 12, in get_trial
    async with session.get(url, headers=headers) as response:
  File "/home/levin/repros/reward-network-iii/.venv/lib/python3.10/site-packages/aiohttp/client.py", line 1194, in __aenter__
    self._resp = await self._coro
  File "/home/levin/repros/reward-network-iii/.venv/lib/python3.10/site-packages/aiohttp/client.py", line 605, in _request
    await resp.start(conn)
  File "/home/levin/repros/reward-network-iii/.

Loop 1, Started: 51, Finished: 2, Remaining: 150, Running: 49, Total: 152


Task exception was never retrieved
future: <Task finished name='Task-11011' coro=<run_with_limit() done, defined at /tmp/ipykernel_980350/2588555149.py:14> exception=ServerDisconnectedError('Server disconnected')>
Traceback (most recent call last):
  File "/tmp/ipykernel_980350/2588555149.py", line 16, in run_with_limit
    new_trials = await run_participant()
  File "/tmp/ipykernel_980350/4172783153.py", line 8, in run_participant
    trial, get_response_time = await get_trial(prolific_id, experiment_type)
  File "/tmp/ipykernel_980350/1537036795.py", line 12, in get_trial
    async with session.get(url, headers=headers) as response:
  File "/home/levin/repros/reward-network-iii/.venv/lib/python3.10/site-packages/aiohttp/client.py", line 1194, in __aenter__
    self._resp = await self._coro
  File "/home/levin/repros/reward-network-iii/.venv/lib/python3.10/site-packages/aiohttp/client.py", line 605, in _request
    await resp.start(conn)
  File "/home/levin/repros/reward-network-iii/.

Loop 1, Started: 52, Finished: 3, Remaining: 149, Running: 49, Total: 152


...

<Response [200]>
Loop 17, Started: 0, Finished: 0, Remaining: 2, Running: 0, Total: 2
Loop 17, Started: 1, Finished: 0, Remaining: 2, Running: 1, Total: 2


Task exception was never retrieved
future: <Task finished name='Task-29818' coro=<run_with_limit() done, defined at /tmp/ipykernel_980350/2588555149.py:14> exception=ServerDisconnectedError('Server disconnected')>
Traceback (most recent call last):
  File "/tmp/ipykernel_980350/2588555149.py", line 16, in run_with_limit
    new_trials = await run_participant()
  File "/tmp/ipykernel_980350/4172783153.py", line 8, in run_participant
    trial, get_response_time = await get_trial(prolific_id, experiment_type)
  File "/tmp/ipykernel_980350/1537036795.py", line 12, in get_trial
    async with session.get(url, headers=headers) as response:
  File "/home/levin/repros/reward-network-iii/.venv/lib/python3.10/site-packages/aiohttp/client.py", line 1194, in __aenter__
    self._resp = await self._coro
  File "/home/levin/repros/reward-network-iii/.venv/lib/python3.10/site-packages/aiohttp/client.py", line 605, in _request
    await resp.start(conn)
  File "/home/levin/repros/reward-network-iii/.

<Response [200]>
Loop 18, Started: 0, Finished: 0, Remaining: 2, Running: 0, Total: 2
Loop 18, Started: 1, Finished: 0, Remaining: 2, Running: 1, Total: 2
<Response [200]>
Loop 19, Started: 0, Finished: 0, Remaining: 2, Running: 0, Total: 2
Loop 19, Started: 1, Finished: 0, Remaining: 2, Running: 1, Total: 2
<Response [200]>


Task exception was never retrieved
future: <Task finished name='Task-29887' coro=<run_with_limit() done, defined at /tmp/ipykernel_980350/2588555149.py:14> exception=ServerDisconnectedError('Server disconnected')>
Traceback (most recent call last):
  File "/tmp/ipykernel_980350/2588555149.py", line 16, in run_with_limit
    new_trials = await run_participant()
  File "/tmp/ipykernel_980350/4172783153.py", line 8, in run_participant
    trial, get_response_time = await get_trial(prolific_id, experiment_type)
  File "/tmp/ipykernel_980350/1537036795.py", line 12, in get_trial
    async with session.get(url, headers=headers) as response:
  File "/home/levin/repros/reward-network-iii/.venv/lib/python3.10/site-packages/aiohttp/client.py", line 1194, in __aenter__
    self._resp = await self._coro
  File "/home/levin/repros/reward-network-iii/.venv/lib/python3.10/site-packages/aiohttp/client.py", line 605, in _request
    await resp.start(conn)
  File "/home/levin/repros/reward-network-iii/.

Loop 20, Started: 0, Finished: 0, Remaining: 1, Running: 0, Total: 1
<Response [200]>
